In [1]:
import os
from __future__ import print_function, division
import pandas as pd
from datetime import timedelta
from time import strftime
import numpy as np

path = '/home/nilm/Desktop/energydisagg/data' # multi_group
os.chdir(path)

In [2]:
"""
    prepare training data from 'multi_group', which can be downloaded from google drive
    collection : data_in_dictionary, training data
    train_builing : IdList, the building for training
    bprob = IdProbility, the probabilities for the training builings. 
    activation_prob = ActivationProbility, the probabilities for the activations
    
""" 

"\n    prepare training data from 'multi_group', which can be downloaded from google drive\n    collection : data_in_dictionary, training data\n    train_builing : IdList, the building for training\n    bprob = IdProbility, the probabilities for the training builings. \n    activation_prob = ActivationProbility, the probabilities for the activations\n    \n"

In [3]:
train_builing = [1, 14, 19, 28, 39, 41, 51]
channels = ['main','fridge','bottle warmer']
bprob = []
collection = {} # training data
activation_prob = {} #  ActivationProbility

In [4]:
# prepare the related config
for item in sorted(train_builing):
    pathfile = os.path.join(path,'multi_group',str(item))
    aprob = []
    ActivationsList = os.listdir(pathfile)
    ActivationCollection = {}

    for activation in ActivationsList:
        activation_data = pd.read_csv(pathfile+'/'+activation, index_col=0)
        #print(len(activation_data))
        aprob.append(len(activation_data))
        ActivationCollection[str(activation[:-15])]=activation_data
    bprob.append(sum(aprob))
    collection['id_'+str(item)] =  ActivationCollection
    activation_prob['id_'+ str(item)] = [i/sum(aprob) for i in aprob]
building_prob = [i/sum(bprob) for i in bprob]

In [5]:
class Sequence(object):
    """
    Attributes
    ----------
    input : np.ndarray
    target : np.ndarray
    all_appliances : pd.DataFrame
        Column names are the appliance names.
    metadata : dict
    weights : np.ndarray or None
    """
    def __init__(self, shape, target_channels_in_list):
        self.input = np.zeros(shape, dtype=np.float32)
        self.target = {}
        for target_channel in target_channels_in_list:
            self.target[str(target_channel)] = np.zeros(shape, dtype=np.float32)
       


In [6]:
# the RealSource, which will replace the original one
class RealSource(object):
    
    def __init__(self, data_in_dictionary, target_channels_in_list, seq_length, IdList, IdProbility, ActivationProbility,
                num_seq_per_batch=32):
        self.data_in_dictionary = data_in_dictionary
        self.target_channels_in_list = target_channels_in_list
        self.seq_length = seq_length
        self.IdList = IdList
        self.IdProbility = IdProbility
        self.ActivationProbility = ActivationProbility
        self.num_seq_per_batch = num_seq_per_batch
    
    def _select_building(self, IdList, IdProbility):
        """
        For Example:
        
        _select_building(train_builing, building_prob), where
        train_builing = [1, 14, 19]
        building_prob = [0.5, 0.2, 0.3]
        """
        return np.random.choice(IdList, 1, p=IdProbility)
    
    def _select_activation(self, ActivationList, ActivationProbility):
        """
        For Example:
        
        _select_activation(range(len(activation_prob['id_1'])), activation_prob['id_1']), where
        range(len(activation_prob['id_1'])) = [0,1,2,3,5]
        activation_prob['id_1'] = [0.1, 0.1, 0.3, 0.2, 0.3]
        """
        return np.random.choice(ActivationList, 1, p=ActivationProbility)
    
    def get_seq_and_check(self, data, IdList, IdProbility, ActivationProbility):
        """
        get a batch of data
        For Example:
        get_seq_and_check(collection, train_builing, building_prob, activation_prob)
        collection = data
        train_builing = [1, 14, 19]
        building_prob = [0.5, 0.2, 0.3]
        activation_prob = {'id_1':[0.1, 0.1, 0.3, 0.2, 0.3], 
                            'id_14':[0.1, 0.1, 0.3, 0.2, 0.3],
                            'id_19':[0.1, 0.1, 0.3, 0.2, 0.3]}
        
        Warning:
        ------------------------------------------------------------------------
            Currently, setting max_iter == 120, the gap within select_start and end is self.seq_length*2 points
            If the gap is self.seq_length points, it will not success. The cause needs to be figured out
            In the prototype stage, using main as target
        """
        success_for_enough_data = False
        max_iter_for_enough_data = 0
        while not success_for_enough_data:
            max_iter_for_enough_data +=1
            select_building = self._select_building(IdList, IdProbility)[0]
            select_building = 'id_'+str(select_building)
            activation_prob_for_the_select_building = ActivationProbility[select_building ]
            select_activation = self._select_activation(range(len(activation_prob_for_the_select_building)), 
                                           activation_prob_for_the_select_building)[0]      
            get_seq_before_check = data[select_building][str(select_activation)]
            # double check that the index is datetime format
            get_seq_before_check.index = pd.to_datetime(get_seq_before_check.index)
            if len(get_seq_before_check)>=self.seq_length or  max_iter_for_enough_data >= 32 :
                 success_for_enough_data = True
      
        success = False
        max_iter = 0
        while not success:
            max_iter +=1
            select_start = get_seq_before_check.sample(n=1).index[0]
            end = select_start + timedelta(seconds = 60*self.seq_length*2) 
            if len(get_seq_before_check[select_start:end])>=self.seq_length or max_iter==120:
                success = True
                get_seq_after_check = get_seq_before_check[select_start:end]
        
        if max_iter==120:
            seq = None

        else:
            del get_seq_before_check
            #seq = Sequence(self.seq_length)
            seq = Sequence(self.seq_length, self.target_channels_in_list)
            seq.input = np.array(get_seq_after_check[self.target_channels_in_list[0]].values[:self.seq_length])
            #for target in self.target_channels_in_list[1:]:
            #    seq.target = np.array(get_seq_after_check[target].values[:self.seq_length])
            for target in self.target_channels_in_list[1:]:
                seq.target[str(target)] = np.array(get_seq_after_check[target].values[:self.seq_length])
        return seq
    
    def _get_sequence(self):
        seq=self.get_seq_and_check(data = self.data_in_dictionary, 
                                   IdList = self.IdList, 
                                   IdProbility = self.IdProbility, 
                                   ActivationProbility = self.ActivationProbility)
        return seq
    
    def get_batch(self):
        """
        Returns
        -------
        A Batch object or None if source iterator has hit a StopIteration.
        """

        input_sequences = []
        target_sequences = {}
        none_happened = False
        for target in self.target_channels_in_list[1:]:
                target_sequences[str(target)] = []

        for i in range(self.num_seq_per_batch):
            seq = self._get_sequence()
            
            if seq is None:
                none_happened = True
            else:
                input_sequences.append(seq.input.reshape(self.seq_length,1))
                #target_sequences.append(seq.target.reshape(self.seq_length,1))
                for target in self.target_channels_in_list[1:]:
                    target_sequences[str(target)].append(seq.target[str(target)].reshape(self.seq_length,1))
                
        if not none_happened:
            input_sequences = np.asarray(input_sequences).reshape(self.num_seq_per_batch,self.seq_length,1)
            #target_sequences = np.asarray(target_sequences).reshape(self.num_seq_per_batch,self.seq_length,1)
            for target in self.target_channels_in_list[1:]:
                target_sequences[str(target)] = np.asarray(target_sequences[str(target)]).reshape(self.num_seq_per_batch,self.seq_length,1)
        else:
            input_sequences = None
            target_sequences = None
            
        return input_sequences, target_sequences

    

In [7]:
class SynSource(RealSource):
    def __init__(self, data_in_dictionary, target_channels_in_list, seq_length, IdList, IdProbility, ActivationProbility,
                num_seq_per_batch=32):
        
        self.data_in_dictionary = data_in_dictionary
        self.target_channels_in_list = target_channels_in_list
        self.seq_length = seq_length
        self.IdList = IdList
        self.IdProbility = IdProbility
        self.ActivationProbility = ActivationProbility
        self.num_seq_per_batch = num_seq_per_batch
        
    def _get_sequence(self):
        seq = Sequence(self.seq_length, self.target_channels_in_list)
   
        for target in self.target_channels_in_list[1:]:
            channel_seq = super(SynSource,self)._get_sequence()
            if channel_seq is None:
                seq.target[str(target)] = np.array(np.zeros(self.seq_length))
            else:
                seq.target[str(target)] = np.array(channel_seq.target[str(target)])
            seq.input += seq.target[str(target)]
            
        return seq
    

In [8]:
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Activation, Reshape, Flatten, Conv1D, MaxPooling1D, Dropout, LSTM, TimeDistributed, Bidirectional
from keras.optimizers import RMSprop


MODEL_CONV_FILTERS = 32
MODEL_CONV_KERNEL_SIZE = 18
MODEL_CONV_STRIDES = 1
MODEL_CONV_PADDING = 'same'



seq_length = 60

# conv
x = Input(shape=(60,1))
conv_1 = Conv1D(filters=MODEL_CONV_FILTERS, kernel_size=MODEL_CONV_KERNEL_SIZE, padding=MODEL_CONV_PADDING, activation='relu')(x)
drop_1 = Dropout(0.12)(conv_1)

conv_2 = Conv1D(filters=64, kernel_size=12, padding=MODEL_CONV_PADDING, activation='relu')(drop_1)
drop_2 = Dropout(0.14)(conv_2)

conv_3 = Conv1D(filters=128, kernel_size=7, padding=MODEL_CONV_PADDING, activation='relu')(drop_2)
pool_3 = MaxPooling1D(pool_size=2)(conv_3)
drop_3 = Dropout(0.18)(pool_3)

conv_4 = Conv1D(filters=128, kernel_size=3, padding=MODEL_CONV_PADDING, activation='relu')(drop_3)
pool_4 = MaxPooling1D(pool_size=2)(conv_4)
drop_4 = Dropout(0.2)(pool_4)

# reshape
flat_4 = Flatten()(drop_4)

dense_5 = Dense(1280, activation='relu')(flat_4)
drop_5 = Dropout(0.16)(dense_5)

dense_6 = Dense(960, activation='relu')(drop_5)
drop_6 = Dropout(0.14)(dense_6)

dense_7 = Dense(720, activation='relu')(drop_6)
drop_7 = Dropout(0.12)(dense_7)

reshape_8 = Reshape(target_shape=(seq_length, 12))(drop_7)

outputs_disaggregation = []

for appliance_name in channels[1:]:
    biLSTM_1 = Bidirectional(LSTM(6, return_sequences=True))(reshape_8)
    biLSTM_2 = Bidirectional(LSTM(3, return_sequences=True))(biLSTM_1)
    outputs_disaggregation.append(TimeDistributed(Dense(1, activation='relu'), name=appliance_name.replace(" ", "_"))(biLSTM_2))

model = Model(inputs=x, outputs=outputs_disaggregation)
optimizer = RMSprop(lr=0.001, clipnorm=4)
model.compile(optimizer=optimizer, loss='mse', metrics=['mae', 'mse'])




Using TensorFlow backend.


In [9]:
real = RealSource(data_in_dictionary = collection, 
                  target_channels_in_list = channels, 
                  seq_length=60, 
                  IdList = train_builing, 
                  IdProbility = building_prob,
                  ActivationProbility = activation_prob)


In [ ]:
syn = SynSource(data_in_dictionary = collection, 
                  target_channels_in_list = channels, 
                  seq_length=60, 
                  IdList = train_builing, 
                  IdProbility = building_prob,
                  ActivationProbility = activation_prob)

In [ ]:
for i in range(10000):
    if i%2==0:
        main, target = real.get_batch()
        while main is None or target is None:
            main, target = real.get_batch()
    else:
        main, target = syn.get_batch()
        while main is None or target is None:
            main, target = syn.get_batch()
        
    train_metrics = model.train_on_batch(x=main, y=[target[channels[1]], target[channels[2]]])
    
    if i % 100 == 0:
        print('Step : {} , Time : {}\n'.format(i,strftime('%Y-%m-%d_%H_%M')))
        for i, metrics_name in enumerate(model.metrics_names):
            print('{}={:.4f}, '.format(metrics_name, train_metrics[i]), end='')


Step : 0 , Time : 2018-06-22_12_24

loss=15623.1982, fridge_loss=4261.1982, bottle_warmer_loss=11362.0000, fridge_mean_absolute_error=38.9817, fridge_mean_squared_error=4261.1978, bottle_warmer_mean_absolute_error=44.2301, bottle_warmer_mean_squared_error=11361.9990, Step : 100 , Time : 2018-06-22_12_25

loss=6552.0029, fridge_loss=3700.3181, bottle_warmer_loss=2851.6846, fridge_mean_absolute_error=38.9985, fridge_mean_squared_error=3700.3174, bottle_warmer_mean_absolute_error=27.4462, bottle_warmer_mean_squared_error=2851.6841, Step : 200 , Time : 2018-06-22_12_26

loss=8270.9062, fridge_loss=1964.4570, bottle_warmer_loss=6306.4487, fridge_mean_absolute_error=34.3267, fridge_mean_squared_error=1964.4568, bottle_warmer_mean_absolute_error=21.3720, bottle_warmer_mean_squared_error=6306.4443, Step : 300 , Time : 2018-06-22_12_26

loss=8072.0142, fridge_loss=5476.7817, bottle_warmer_loss=2595.2324, fridge_mean_absolute_error=45.1808, fridge_mean_squared_error=5476.7783, bottle_warmer_mean

In [ ]:
model.save('multi_group_frige_bw_0622.h5')